# Jupyter Widgets Tutorial

## Table of Contents
- [Section 1: Intro](#Section-1:-Intro)
- [Section 2: Traitlets](#Section-2:-Traitlets)
- [Section 3: Widget List](#Section-3:-Widget-List)
- [Section 4: Visualization Libraries](#Section-4:-Visualization-Libraries)
    - [bqplot](#Visualization-Libraries:-bqplot)
    - [IPyVolume](#Visualization-Libraries:-IPyVolume)
    - [IPyLeaflets](#Visualization-Libraries:-IPyLeaflets)
- [Section 5: Demo](#Section-5:-Demo)
    - [Kelp](http://localhost:8889)
    - [Droplet](http://localhost:8889)

# Section 1: Intro

## Why widgets?

- "Reduce the distance between user and data"

## Similar software

- Mathematica
- Plotly
- Matlab?

## Installation

## Overview

# Section 2: Traitlets

- Objects which can react to having values assigned

In [175]:
import traitlets as tr

## Links

In [176]:
class NiceKid(tr.HasTraits):
    best_color = tr.Unicode()
    
    def __init__(self, best_color):
        self.best_color = best_color
        self.friend = None
    
    def make_friends(self, friend):
        if self.friend is not None:
            self.link.unlink()
            
        self.friend = friend
            
        self.link = tr.link(
            (self, 'best_color'),
            (self.friend, 'best_color')
        )
        
    def say_color(self):
        print("My favorite color is {}".format(
            self.best_color
        ))

In [177]:
jim = NiceKid('blue')
jane = NiceKid('green')
jim.say_color()
jane.say_color()

My favorite color is blue
My favorite color is green


In [178]:
jim.make_friends(jane)

jim.best_color = 'red'
jim.say_color()
jane.say_color()

My favorite color is red
My favorite color is red


In [179]:
jane.best_color = 'aquamarine'
jim.say_color()
jane.say_color()

My favorite color is aquamarine
My favorite color is aquamarine


## Directional Links

In [180]:
class MeanKid(tr.HasTraits):
    best_color = tr.Unicode()
    worst_color = tr.Unicode(allow_none=True)
    
    def __init__(self, best_color):
        self.best_color = best_color
        self.worst_color = None
        self.enemy = None
    
    def make_enemy(self,enemy):
        if self.enemy is not None:
            self.link.unlink()
            
        self.enemy = enemy
            
        self.link = tr.dlink(
            (self.enemy, 'best_color'),
            (self, 'worst_color')
        )
        
    def say_color(self):
        print("My favorite color is {}".format(
            self.best_color
        ))
        if self.worst_color is not None:
            print("The worst color is {}!".format(
                self.worst_color
            ))

In [181]:
carl = MeanKid('chartreuse')
carl.say_color()

My favorite color is chartreuse


In [182]:
jim.say_color()

My favorite color is aquamarine


In [183]:
carl.make_enemy(jim)
carl.say_color()

My favorite color is chartreuse
The worst color is aquamarine!


In [184]:
jim.best_color = 'blue'
carl.say_color()

My favorite color is chartreuse
The worst color is blue!


In [185]:
carl.worst_color = 'yellow'
jim.say_color()

My favorite color is blue


## Observe

In [186]:
class Citizen(tr.HasTraits):
    country = tr.Unicode()
    language = tr.Unicode()
    
    language_dict = {
        'England': 'English',
        'Germany': 'German',
    }
    
    def __init__(self, country):
        self.country = country
        self.set_language()
        self.observe(self.set_language, names='country')
        
    def greet(self):
        print("Hello! I'm from {} and I speak {}.".format(
            self.country, 
            self.language
        ))
        
    # tr.HasTraits.observe passes an argument describing the change that occurred
    # We don't need that here, so we'll lump it into *args and ignore it.
    def set_language(self, *args):
        self.language = self.language = self.language_dict[self.country]
        print(args)

In [187]:
fred = Citizen('England')
fred.greet()

()
Hello! I'm from England and I speak English.


In [188]:
fred.country = 'Germany'
fred.greet()

({'type': 'change', 'name': 'country', 'owner': <__main__.Citizen object at 0x7fd23a010710>, 'new': 'Germany', 'old': 'England'},)
Hello! I'm from Germany and I speak German.


# Section 3: Widget List

[Widget List](08b_IPyWidgets-Widget-List.ipynb)

# Section 3.5: IPyWidgets Basics

In [189]:
import ipywidgets as ipw

In [190]:
def f(x):
    return 4*x + 3

In [191]:
ipw.interact(f, x=(1,5));

A Jupyter Widget

# Section 4: Visualization Libraries

# Visualization Libraries: bqplot

- 2D plotting based on d3.js

- Declarative (like ggplot), as opposed to imperative (like matplotlib)

- Traitful, and built with widgets in mind!

In [192]:
import bqplot as bq
from numpy import *

In [193]:
x = linspace(0, 1, 101)
y = sin(4*pi*x)

In [194]:
xscale = bq.LinearScale()
yscale = bq.LinearScale()

xax = bq.Axis(scale=xscale)
yax = bq.Axis(scale=yscale, orientation='vertical')

line = bq.Lines(
    x=x,
    y=y,
    scales={
        'x': xscale,
        'y': yscale
    },
)
bqfig = bq.Figure(
    axes=(xax, yax),
    marks=[line],
    animation_duration=500
)

In [195]:
bqfig

A Jupyter Widget

In [196]:
line.y = cos(4*pi*x)

In [197]:
omega_slider = ipw.IntSlider(
    description=r'$\omega$',
    min=1,
    max=20
)

def update_line(*args):
    omega = omega_slider.value
    line.y = cos(omega * x)

omega_slider.observe(update_line, names='value')

In [198]:
ipw.VBox([
    omega_slider,
    bqfig
])

A Jupyter Widget

# Visualization Libraries: IPyVolume

In [199]:
import ipyvolume as ipv
from numpy import *

In [200]:
x = linspace(0,1,101)
y = linspace(0,1,101)
z = linspace(0,1,101)

In [201]:
X, Y = meshgrid(x, y)
Z1 = exp(Y) * (cos(4*pi*X) + cos(4*pi*Y))
Z2 = exp(X) * (cos(4*pi*(X+1/4)) + cos(4*pi*(Y+1/4)))
ipv.figure()
ipv.plot_surface(X,Y,Z1, color='red')
ipv.plot_surface(X,Y,Z2, color='blue')
ipv.show()

A Jupyter Widget

In [202]:
X,Y,Z = meshgrid(x,y,z)

F = sin(4*pi*X) + sin(4*pi*Y) + Z

box = ipv.quickvolshow(F)
controls, fig = box.children
box

A Jupyter Widget

In [203]:
fig.volume_data = fig.volume_data / Z

/home/oliver/local/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/home/oliver/local/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


# Visualization Libraries: IPyLeaflet

In [204]:
import ipyleaflet as ipl
import json

In [205]:
center = [42.276880107831104, -83.73809337615967]
zoom = 17
m = ipl.Map(center=center, zoom=zoom)
m

A Jupyter Widget

In [206]:
m.interact(zoom=(5,20,1))

A Jupyter Widget

In [207]:
m

A Jupyter Widget

In [208]:
mark = ipl.Marker(location=m.center)
m += mark

In [209]:
mark.interact(opacity=(0.0,1.0,0.01))

A Jupyter Widget

# Demos

- My use cases from my research
- These involve a lot of moving parts, so best to just link to other server where they're running

- [Kelp](http://localhost:8889)
- [Droplet](http://localhost:8889)

# Example Widget

Let's make a widget dashboard with two figures:
1. A volume plot of a function with, say, two parameters
    - A plane through the volume plot
2. A separate 2D plot with contours of the volume data intersected with the plane

In [210]:
class VolumeCutWidget(ipw.VBox):
    def __init__(self, func_3d):
        self.func_3d = func_3d
        
        # Very important! If you don't
        # call VBox's constructor, your
        # widget won't be registered and 
        # it won't render!
        super().__init__()
        
        self.init_vals()
        self.init_elements()
        self.init_layout()
        #self.init_logic()
        self.update_volume_plot()
    
    def init_vals(self):
        # linspaces
        self.x = linspace(0, 1, 101)
        self.y = linspace(0, 1, 101)
        self.z = linspace(0, 1, 101)
        
        # Volume meshgrids
        self.vX, self.vY, self.vZ = meshgrid(
            self.x,
            self.y,
            self.z
        )
        
        # Color meshgrids
        self.cX, self.cY = meshgrid(
            self.x,
            self.z
        )
        
    def init_elements(self):
        self.title = ipw.HTML("<h3>Volume Cut Widget</h3>")
        self.ipv_fig = ipv.figure()
        self.init_bq_fig()
        
    def init_bq_fig(self):
        self.bq_xscale = bq.LinearScale()
        self.bq_yscale = bq.LinearScale()
        self.bq_colorscale = bq.ColorScale()
        self.bq_xax = bq.Axis(scale=self.bq_xscale)
        self.bq_yax = bq.Axis(scale=self.bq_yscale, orientation='vertical')
        self.bq_colorax = bq.ColorAxis(scale=self.bq_colorscale)
        self.bq_heat = bq.HeatMap(
            x=self.x, 
            y=self.z, 
            color=(self.cX + self.cY),
            scales={
                'x': self.bq_xscale,
                'y': self.bq_yscale,
                'color': self.bq_colorscale
            }
            
        )
        self.bq_fig = bq.Figure(
            marks=[self.bq_heat],
            axes=[
                self.bq_xax,
                self.bq_yax,
                self.bq_colorax
            ]
        )
    
    def init_layout(self):
        self.children = [
            self.title,
            ipw.HBox([
                self.ipv_fig,
                self.bq_fig
            ])
        ]
    
    def update_volume_plot(self):
        ipv.volshow(
            self.func_3d(
                self.vX,
                self.vY,
                self.vZ
            )
        )
    
    def plot_plane(self, z):
    ipv.plot_trisurf(
        [0,0,1,1],
        [0,1,0,1],
        [z,z,z,z],
        triangles=[[0, 2, 3], [0, 3, 1]]
    )

IndentationError: expected an indented block (<ipython-input-210-4943c2f24450>, line 87)